In [1]:
#Standard Header used on the projects

#first the major packages used for math and graphing
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import scipy.special as sp

#Custome graph format style sheet
#plt.style.use('Prospectus.mplstyle')

#If being run by a seperate file, use the seperate file's graph format and saving paramaeters
#otherwise set what is needed
if not 'Saving' in locals():
    Saving = False
if not 'Titles' in locals():
    Titles = True
if not 'Ledgends' in locals():
    Ledgends = True
if not 'FFormat' in locals():
    FFormat = '.png'

#Standard cycle to make black and white images and dashed and line styles
default_cycler = (cycler('color', ['0.00', '0.40', '0.60', '0.70']) + cycler(linestyle=['-', '-', '-', '-']))
plt.rc('axes', prop_cycle=default_cycler)
my_cmap = plt.get_cmap('gray')

#Extra Headers:
import os as os
import pywt as py
import statistics as st
import os as os
import random
import multiprocessing
from joblib import Parallel, delayed
import platform

from time import time as ti

#import CoreFunctions as cf
#from skimage.restoration import denoise_wavelet

In [2]:
import pandas as pd

import torch
from gluonts.dataset.multivariate_grouper import MultivariateGrouper
from gluonts.dataset.repository.datasets import dataset_recipes, get_dataset
from pts.model.tempflow import TempFlowEstimator
from pts.model.transformer_tempflow import TransformerTempFlowEstimator
from pts import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import MultivariateEvaluator

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/sciclone/home/dchendrickson01/.conda/envs/torchmpi/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
HostName = platform.node()

if HostName == "Server":
    Computer = "Desktop"   
elif HostName[-6:] == 'wm.edu':
    Computer = "SciClone"
elif HostName == "SchoolLaptop":
    Computer = "LinLap"
elif HostName == "WTC-TAB-512":
    Computer = "PortLap"
else:
    Computer = "WinLap"

if Computer == "SciClone":
    location = '/sciclone/home20/dchendrickson01/image/'
elif Computer == "WinLap":
    location = 'C:\\Data\\'
elif Computer == "Desktop":
    location = "E:\\Backups\\Dan\\CraneData\\"
elif Computer == "LinLap":
    location = '/home/dan/Output/'
elif Computer == 'PortLap':
    location = 'C:\\users\\dhendrickson\\Desktop\\AccelData\\'

if Computer ==  "SciClone":
    rootfolder = '/sciclone/home20/dchendrickson01/'
    folder = '/sciclone/scr10/dchendrickson01/Recordings2/'
    imageFolder = '/sciclone/scr10/dchendrickson01/Move3Dprint/'
elif Computer == "Desktop":
    rootfolder = location
    folder = rootfolder + "Recordings2\\"
elif Computer =="WinLap":
    rootfolder = location
    folder = rootfolder + "Recordings2\\"   
elif Computer == "LinLap":
    rootfolder = '/home/dan/Data/'
    folder = rootfolder + 'Recordings2/'
elif Computer =='PortLap':
    rootfolder = location 
    folder = rootfolder + 'Recordings2\\'

In [4]:
Saving = False
location = folder
Titles = True
Ledgends = True

f = 0
freq = "2s"

In [5]:
files = ['230418 recording1.csv','230419 recording1.csv']#,'230420 recording1.csv','230421 recording1.csv',
         #'230418 recording2.csv','230419 recording2.csv','230420 recording2.csv','230421 recording2.csv']

In [6]:
def MakeDataframe(file):
    dataset = pd.read_table(folder+file, delimiter =", ", header=None, engine='python')

    dataset = dataset.rename(columns={0:"Day"})
    dataset = dataset.rename(columns={1:"Second"})
    dataset = dataset.rename(columns={2:"FracSec"})
    dataset = dataset.rename(columns={3:"x"})
    dataset = dataset.rename(columns={4:"y"})
    dataset = dataset.rename(columns={5:"z"})
    dataset = dataset.rename(columns={6:"Sensor"})

    dataset[['Day','Second']] = dataset[['Day','Second']].apply(lambda x: x.astype(int).astype(str).str.zfill(6))
    dataset[['FracSec']] = dataset[['FracSec']].apply(lambda x: x.astype(int).astype(str).str.zfill(4))

    dataset["timestamp"] = pd.to_datetime(dataset.Day+dataset.Second+dataset.FracSec,format='%y%m%d%H%M%S%f')

    dataset["x"] = dataset.x - np.average(dataset.x)
    dataset["y"] = dataset.y - np.average(dataset.y)
    dataset["z"] = dataset.z - np.average(dataset.z)
    dataset["r"] = np.sqrt(dataset.x**2 + dataset.y**2 + dataset.z**2)

    dataset.index = dataset.timestamp
    
    return dataset

df = []
for file in files:
    tempDF = MakeDataframe(file)
    df.append(tempDF)
dataset = pd.concat(df)[["x","y","z","r"]]

In [7]:
data = Parallel(n_jobs=8)(delayed(MakeDataframe)(file) for file in files)
dataset = pd.concat(data)[["x","y","z","r"]]
dataset.head()

,x,y,z,r
timestamp,,,,
2023-04-18 00:00:00.172800,-0.002305,0.002889,0.004128,0.005541
2023-04-18 00:00:00.177600,0.015265,0.000219,-0.016372,0.022385
2023-04-18 00:00:00.182300,-0.012805,0.003869,0.024888,0.028255
2023-04-18 00:00:00.187000,-0.014025,-0.011261,-0.004172,0.018464
2023-04-18 00:00:00.191700,0.005495,0.008269,0.006328,0.011774


In [8]:
from gluonts.dataset.split import OffsetSplitter, DateSplitter

In [9]:
splitter = DateSplitter(
    date=pd.Period('2023-04-18', freq='500ms'))

train, test_template = splitter.split(dataset)

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [10]:
test_template

TestTemplate(dataset=                                   x         y         z         r
timestamp                                                         
2023-04-18 00:00:00.172800 -0.002305  0.002889  0.004128  0.005541
2023-04-18 00:00:00.177600  0.015265  0.000219 -0.016372  0.022385
2023-04-18 00:00:00.182300 -0.012805  0.003869  0.024888  0.028255
2023-04-18 00:00:00.187000 -0.014025 -0.011261 -0.004172  0.018464
2023-04-18 00:00:00.191700  0.005495  0.008269  0.006328  0.011774
...                              ...       ...       ...       ...
2023-04-19 23:59:59.983200 -0.010660 -0.014843  0.023773  0.029985
2023-04-19 23:59:59.987900 -0.010660  0.020057 -0.000147  0.022714
2023-04-19 23:59:59.992600 -0.010660 -0.004343 -0.007227  0.013591
2023-04-19 23:59:59.997300 -0.003090  0.004187  0.007173  0.008861
2023-04-20 00:00:00.002000 -0.001620  0.009317 -0.010647  0.014241

[37557076 rows x 4 columns], splitter=DateSplitter(date=Period('2023-04-18 00:00:00.000', '500L')))

In [11]:
test_dataset = test_template.generate_instances(
    prediction_length=7,
    windows=2,
    distance=3, # windows are three time steps apart from each other
)


In [12]:
test_dataset

TestData(dataset=                                   x         y         z         r
timestamp                                                         
2023-04-18 00:00:00.172800 -0.002305  0.002889  0.004128  0.005541
2023-04-18 00:00:00.177600  0.015265  0.000219 -0.016372  0.022385
2023-04-18 00:00:00.182300 -0.012805  0.003869  0.024888  0.028255
2023-04-18 00:00:00.187000 -0.014025 -0.011261 -0.004172  0.018464
2023-04-18 00:00:00.191700  0.005495  0.008269  0.006328  0.011774
...                              ...       ...       ...       ...
2023-04-19 23:59:59.983200 -0.010660 -0.014843  0.023773  0.029985
2023-04-19 23:59:59.987900 -0.010660  0.020057 -0.000147  0.022714
2023-04-19 23:59:59.992600 -0.010660 -0.004343 -0.007227  0.013591
2023-04-19 23:59:59.997300 -0.003090  0.004187  0.007173  0.008861
2023-04-20 00:00:00.002000 -0.001620  0.009317 -0.010647  0.014241

[37557076 rows x 4 columns], splitter=DateSplitter(date=Period('2023-04-18 00:00:00.000', '500L')), prediction_

In [13]:
evaluator = MultivariateEvaluator(quantiles=(np.arange(20)/20.0)[1:],
                                  target_agg_funcs={'sum': np.sum})


In [14]:
estimator = TempFlowEstimator(
    target_dim=4,
    prediction_length=test_dataset.prediction_length,
    cell_type='GRU',
    input_size=552,
    freq="1T",
    scaling=True,
    dequantize=True,
    n_blocks=4,
    trainer=Trainer(device=device,
                    epochs=45,
                    learning_rate=1e-3,
                    num_batches_per_epoch=100,
                    batch_size=64)
)

In [16]:
predictor = estimator.train(train)
forecast_it, ts_it = make_evaluation_predictions(dataset=dataset_test,
                                             predictor=predictor,
                                             num_samples=100)
forecasts = list(forecast_it)
targets = list(ts_it)

agg_metric, _ = evaluator(targets, forecasts, num_series=len(dataset_test))

ValueError: prefetch_factor option could only be specified in multiprocessing.let num_workers > 0 to enable multiprocessing, otherwise set prefetch_factor to None.